In [1]:
import torch
import torch.nn as nn
torch.cuda.is_available()
import pandas as pd
import matplotlib2tikz

In [2]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS, PUCK_SQUARE_DISTANCES, PUCK_ANGLE_FEATURES
from isaac.sanity import class_proportions
from isaac.models import initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np
import matplotlib2tikz

In [3]:
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 25
STEP_SIZE = 1
SEQ_END = 1800

In [4]:
ALL_COLS = BASIC_TRAINING_COLS + PUCK_SQUARE_DISTANCES + PUCK_ANGLE_FEATURES

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=ALL_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=500, cols=ALL_COLS)

100%|██████████| 500/500 [00:05<00:00, 98.22it/s] 


# MASS TRAINING

In [7]:
INPUT_DIM = None    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

In [8]:
labels = ["distances+angles", "distances", "angles", "baseline"]
stats_dfs = []

for tr_features, label in zip([ALL_COLS, BASIC_TRAINING_COLS+PUCK_SQUARE_DISTANCES, 
                               BASIC_TRAINING_COLS+PUCK_ANGLE_FEATURES, BASIC_TRAINING_COLS], labels):
    
    loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=MASS_CLASS_COLS, 
                                      training_columns=tr_features, batch_size=BATCH_SIZE, 
                                      normalise_data=NORMALISE_DATA)
        
    network_params = (len(tr_features), HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["features", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_end=SEQ_END,
                                                                   step_size=STEP_SIZE)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["features"] = label
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
# stats.to_hdf("features_plots/mass_stats.h5", key="stats")

100%|██████████| 500/500 [00:00<00:00, 929.83it/s] 
Train_loss (0.80)	 Train_acc (67.17)	 Val_acc (35.40): 100%|██████████| 25/25 [08:32<00:00, 20.66s/it]
Train_loss (0.85)	 Train_acc (64.00)	 Val_acc (39.20): 100%|██████████| 25/25 [08:31<00:00, 20.52s/it]
Train_loss (0.92)	 Train_acc (55.83)	 Val_acc (39.60): 100%|██████████| 25/25 [08:34<00:00, 20.60s/it]
100%|██████████| 500/500 [00:00<00:00, 1092.63it/s]
Train_loss (1.04)	 Train_acc (45.34)	 Val_acc (40.00): 100%|██████████| 25/25 [08:34<00:00, 20.76s/it]
Train_loss (0.83)	 Train_acc (63.89)	 Val_acc (49.40): 100%|██████████| 25/25 [08:42<00:00, 20.99s/it]
Train_loss (0.72)	 Train_acc (67.09)	 Val_acc (54.40): 100%|██████████| 25/25 [08:48<00:00, 21.15s/it]
100%|██████████| 500/500 [00:00<00:00, 941.07it/s]
Train_loss (1.08)	 Train_acc (40.29)	 Val_acc (40.40): 100%|██████████| 25/25 [08:42<00:00, 20.84s/it]
Train_loss (0.94)	 Train_acc (56.94)	 Val_acc (36.80): 100%|██████████| 25/25 [08:45<00:00, 21.00s/it]
Train_loss (0.86)	 Tr

# FORCE TRAINING

In [6]:
INPUT_DIM = None    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

In [7]:
labels = ["distances+angles", "distances", "angles", "baseline"]
stats_dfs = []

for tr_features, label in zip([ALL_COLS, BASIC_TRAINING_COLS+PUCK_SQUARE_DISTANCES, 
                               BASIC_TRAINING_COLS+PUCK_ANGLE_FEATURES, BASIC_TRAINING_COLS], labels):
    
    loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=FORCE_CLASS_COLS, 
                                      training_columns=tr_features, batch_size=BATCH_SIZE, 
                                      normalise_data=NORMALISE_DATA)
        
    network_params = (len(tr_features), HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["features", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_end=SEQ_END,
                                                                   step_size=STEP_SIZE)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["features"] = label
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf("features_plots/force_stats.h5", key="stats")

100%|██████████| 500/500 [00:00<00:00, 945.51it/s] 
Train_loss (0.59)	 Train_acc (75.26)	 Val_acc (53.20): 100%|██████████| 25/25 [08:51<00:00, 21.32s/it]
Train_loss (0.58)	 Train_acc (74.60)	 Val_acc (52.40): 100%|██████████| 25/25 [08:59<00:00, 21.70s/it]
Train_loss (0.58)	 Train_acc (75.77)	 Val_acc (52.60): 100%|██████████| 25/25 [08:58<00:00, 21.47s/it]
100%|██████████| 500/500 [00:00<00:00, 1064.65it/s]
Train_loss (0.61)	 Train_acc (72.51)	 Val_acc (53.60): 100%|██████████| 25/25 [08:59<00:00, 21.65s/it]
Train_loss (0.62)	 Train_acc (71.63)	 Val_acc (53.80): 100%|██████████| 25/25 [09:01<00:00, 21.79s/it]
Train_loss (0.58)	 Train_acc (70.00)	 Val_acc (53.80): 100%|██████████| 25/25 [09:03<00:00, 21.66s/it]
100%|██████████| 500/500 [00:00<00:00, 902.05it/s]
Train_loss (0.58)	 Train_acc (73.71)	 Val_acc (50.20): 100%|██████████| 25/25 [09:05<00:00, 21.84s/it]
Train_loss (0.63)	 Train_acc (67.40)	 Val_acc (50.40): 100%|██████████| 25/25 [09:05<00:00, 21.80s/it]
Train_loss (0.63)	 Tr